In [18]:
# Multi-Armed Bandit problem
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim

In [19]:
# init our agent

tf.reset_default_graph()

# feedforward
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

# training
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])

loss = -(tf.log(responsible_weight) * reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
update = optimizer.minimize(loss)

In [20]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        # List our bandits. Arms 4, 2, and 1 are the most optimal right now.
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0,len(self.bandits)) #Returns a random state for each episode.
        return self.state
        
    def pullArm(self,action):
        # A random number.
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            #give a positive reward.
            return 1
        else:
            #give a negative reward.
            return -1

In [21]:
class agent():
    def __init__(self, lr, s_size,a_size):
        # feed forward net
        self.state_in= tf.placeholder(shape=[1],dtype=tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in,s_size)
        output = slim.fully_connected(state_in_OH,a_size,\
            biases_initializer=None,activation_fn=tf.nn.sigmoid,weights_initializer=tf.ones_initializer())
        self.output = tf.reshape(output,[-1])
        self.chosen_action = tf.argmax(self.output,0)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
        self.responsible_weight = tf.slice(self.output,self.action_holder,[1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

In [22]:
tf.reset_default_graph() #Clear the Tensorflow graph.

cBandit = contextual_bandit() 
myAgent = agent(lr=0.001,s_size=cBandit.num_bandits,a_size=cBandit.num_actions) #Load the agent.
weights = tf.trainable_variables()[0] 

total_episodes = 10000 
total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions]) 
e = 0.1 

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit() 
        
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action,feed_dict={myAgent.state_in:[s]})
        
        reward = cBandit.pullArm(action) 
        
        feed_dict={myAgent.reward_holder:[reward],myAgent.action_holder:[action],myAgent.state_in:[s]}
        _,ww = sess.run([myAgent.update,weights], feed_dict=feed_dict)
        
        total_reward[s,action] += reward
        if i % 500 == 0:
            print("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1)))
        i+=1
for a in range(cBandit.num_bandits):
    print("The agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising....")
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print("...and it was right!")
    else:
        print("...and it was wrong!")

Mean reward for each of the 3 bandits: [0.25 0.   0.  ]
Mean reward for each of the 3 bandits: [20.75 37.25 34.25]
Mean reward for each of the 3 bandits: [54.25 74.75 75.25]
Mean reward for each of the 3 bandits: [ 92.   111.75 109.  ]
Mean reward for each of the 3 bandits: [127.5  150.   145.75]
Mean reward for each of the 3 bandits: [168.25 182.75 183.75]
Mean reward for each of the 3 bandits: [203.5  224.25 217.  ]
Mean reward for each of the 3 bandits: [242.75 262.75 251.75]
Mean reward for each of the 3 bandits: [282.5  302.75 286.5 ]
Mean reward for each of the 3 bandits: [324.   337.   324.25]
Mean reward for each of the 3 bandits: [358.25 381.5  360.  ]
Mean reward for each of the 3 bandits: [398.25 417.25 395.25]
Mean reward for each of the 3 bandits: [435.75 458.   433.  ]
Mean reward for each of the 3 bandits: [470.25 502.25 463.75]
Mean reward for each of the 3 bandits: [511.5  538.75 499.  ]
Mean reward for each of the 3 bandits: [549.   584.5  530.75]
Mean reward for each